# TCGA 사이트

https://cancergenome.nih.gov/    

## Project별 데이터 현황 

https://gdc-portal.nci.nih.gov/projects/t


![](01_자료수집_01.jpg)


- Seq와 Exp 데이터 수가 100개 이하인  ARGET-OS, TCGA-KICH, TCGA-ACC, TCGA-MESO, TCGA-UVM, TARGET-RT, TCGA-DLBC, TCGA-UCS, TCGA-CHOL, TARGET-CCSK 프로젝트이며,
- Seq와 Exp 데이터 수가 0개인 TARGET-OS, TARGET-CCSK가 있음.

## TCGA 데이터 파일 구조 

- https://gdc-portal.nci.nih.gov/search/s?facetTab=cases  에서 Manifest 파일 다운로드와  Files 탭의 JSON 포맷으로 Export table을 함.
- gdc_manifest.2017-03-13T05-24-49.083790_ALL.json
- gdc_manifest.2017-03-13T05-24-49.083790_ALL.tsv

In [1]:
gdc_manifest <- read.csv("gdc_manifest.2017-03-13T05-24-49.083790_ALL.tsv", sep="\t")

In [7]:
head(gdc_manifest , n=1)
summary(gdc_manifest)

,id,filename,md5,size,state
1,8bfeb962-e2b3-463a-9fbb-f5c6296f3ec2,8bfeb962-e2b3-463a-9fbb-f5c6296f3ec2.vep.reheader.vcf.gz,ace85792006140a9b4895976ecba725e,378676,submitted


                                    id        
 0000047f-772f-4241-980d-1f667686fe60:     1  
 00005051-36c7-4850-9e2c-243be54077ea:     1  
 00006c36-08ae-4f93-a580-1b798d153d7c:     1  
 00007ccc-269b-4cd0-a0b1-6e5d700a8e5f:     1  
 00008523-edd6-456e-81aa-d1e4aee3ecf9:     1  
 0000bb29-0e74-4b06-9114-308d489f3ea6:     1  
 (Other)                             :274815  
                        filename                                    md5        
 isoforms.quantification.txt: 11488   00003ef5075ad992c45187773c6fdac2:     1  
 mirnas.quantification.txt  : 11488   0000bf2dba764eca3f3a6d2395cf1e59:     1  
 131120.bam                 :    20   0000d0d067a34113bcc509e602ffadd2:     1  
 131121.bam                 :    18   00012f5a51f6c50318d86637ccf09ad9:     1  
 131027.bam                 :    17   00017a088f77fbae422569b5510f1fa2:     1  
 125807.bam                 :    16   0001e044aae9ce17bc8bd2be7a22e4c0:     1  
 (Other)                    :251774   (Other)                   

In [10]:
sum( gdc_manifest$size  ) / 1024 / 1024 / 1024 /  1024

[1] 427.9847

## TCGA 데이터 파일을 DB에 저장하기 

MYSQL 스키마 구조 

```
use mysql;

CREATE DATABASE tcga_gdc DEFAULT character set utf8 ;

GRANT ALL privileges ON tcga_gdc.* TO gdc@localhost  IDENTIFIED BY 'gdcPW!@#';
GRANT ALL privileges ON tcga_gdc.* TO gdc@'%'  IDENTIFIED BY 'gdcPW!@#';
flush privileges;

use tcga_gdc;


DROP TABLE IF EXISTS gdc_files CASCADE ;
CREATE TABLE IF NOT EXISTS gdc_files (
  id           VARCHAR(40)  PRIMARY KEY,
  filename     VARCHAR(128),
  md5          VARCHAR(40),
  size         BIGINT ,
  state        VARCHAR(10)
);
CREATE INDEX idx_gdc_files_filename ON gdc_files(filename); 


DROP TABLE IF EXISTS gdc_manifest CASCADE ;
CREATE TABLE IF NOT EXISTS gdc_manifest (
  seq_id        INT NOT NULL AUTO_INCREMENT,
  file_name     VARCHAR(128),
  project_id    VARCHAR(40),
  case_id       VARCHAR(40),
  data_format   VARCHAR(10),
  access        VARCHAR(10),
  data_category VARCHAR(40),
  file_size     BIGINT,
  PRIMARY KEY ( seq_id )
);
CREATE INDEX idx_gdc_manifest_file_name     ON gdc_manifest(file_name); 
CREATE INDEX idx_gdc_manifest_project_id    ON gdc_manifest(project_id); 
CREATE INDEX idx_gdc_manifest_case_id       ON gdc_manifest(case_id); 
CREATE INDEX idx_gdc_manifest_data_format   ON gdc_manifest(data_format); 
CREATE INDEX idx_gdc_manifest_data_category ON gdc_manifest(data_category); 
```

- https://dev.mysql.com/downloads/connector/python/?os=31 에서  mysql-connector-python-2.1.5-1.el7.x86_64.rpm 다운로드

sudo yum install -y mysql-connector-python-2.1.5-1.el7.x86_64.rpm


In [ ]:
#  mysql_python.py

import mysql.connector
from mysql.connector import MySQLConnection, Error

def connect():
    """ Connect to MySQL database """
    try:
        conn = mysql.connector.connect(host='localhost',database='tcga_gdc',user='gdc',password='gdcPW!@#')
        if conn.is_connected():
            print('Connected to MySQL database')

    except Error as e:
        print(e)

    return conn


def insert_gdc_files(file_infos):
    query = "INSERT INTO gdc_files(id, filename, md5, size, state) VALUES( %s , %s ,%s , %s , %s ) "
    try:
        conn =  connect()

        cursor = conn.cursor()
        cursor.executemany(query, file_infos)

        conn.commit()
    except Error as e:
        print('Error:', e)

    finally:
        cursor.close()
        conn.close()

def insert_gdc_manifest(manifest):
    query = "INSERT INTO gdc_manifest(file_name, project_id, case_id, data_format, access, data_category, file_size) " \
            "VALUES(%s,%s,%s,%s,%s,%s,%s)"

    try:
        conn = connect()

        cursor = conn.cursor()
        cursor.executemany(query, manifest )

        conn.commit()
    except Error as e:
        print('Error:', e)

    finally:
        cursor.close()
        conn.close()

In [ ]:
#  insert_gdc_files.py 
from  mysql_python import insert_gdc_files

FETCH_SIZE = 100

with open('gdc_manifest.2017-03-13T05-24-49.083790_ALL.tsv','r') as gdc :
    cnt = 0
    head_skip = True
    in_data = []

    for line in gdc:
        cnt += 1
        if( head_skip ) :
            head_skip = False
            continue;

        #print( line )
        rows = line.strip().split('\t')
        in_data.append( (rows[0],rows[1],rows[2],int(rows[3]),rows[4])  )

        if( cnt % FETCH_SIZE == 0) :
            insert_gdc_files( in_data )
            in_data = []


    insert_gdc_files( in_data )


In [ ]:
#  insert_gdc_manifest.py
from  mysql_python import insert_gdc_manifest
import json

FETCH_SIZE = 100
with open('gdc_manifest.2017-03-13T05-24-49.083790_ALL.json') as data_file:
    data = json.load(data_file)

cnt = 0
in_data = []
for obj  in data :
    cnt += 1
    in_data.append( (obj['file_name'], obj['cases'][0]['project']['project_id'], obj['cases'][0]['case_id'],
                     obj['data_format'], obj['access'], obj['data_category'], obj['file_size'])  )
    if( cnt % FETCH_SIZE == 0) :
        insert_gdc_manifest( in_data )
        print( "insert_gdc_manifest() size=", cnt  )
        in_data = []

insert_gdc_manifest( in_data )

## mRNA-Seq 데이터 파일만 가져오기

- https://docs.gdc.cancer.gov/Data/PDF/Data_UG.pdf
    
![](01_자료수집_02.jpg)

![](01_자료수집_03.jpg)

- 최종 정제된 파일인 UQ-FPKM 임을 알 수 있고, 이 데이터만들 추출하는 SQL문

```
SELECT f.id, f.filename, m.project_id 
FROM gdc_files f INNER JOIN  gdc_manifest  m
WHERE 
    f.filename = m.file_name AND 
    f.size = m.file_size AND
    m.file_name <> 'mirnas.quantification.txt' AND
    m.file_name <> 'isoforms.quantification.txt' AND
    m.file_name like '%FPKM-UQ.txt.gz' AND
    m.access = 'open' AND
    m.data_category = 'Transcriptome Profiling' AND
    m.project_id =  'TCGA-CHOL'      
ORDER BY m.project_id , f.filename 

```

In [ ]:
# download_gdc.py
import subprocess
import shlex
import os
from  mysql_python import query

sql =  """
SELECT f.id, f.filename, m.project_id
FROM gdc_files f INNER JOIN  gdc_manifest  m
WHERE
    f.filename = m.file_name AND
    f.size = m.file_size AND
    m.file_name <> 'mirnas.quantification.txt' AND
    m.file_name <> 'isoforms.quantification.txt' AND
    m.file_name like '%FPKM-UQ.txt.gz' AND
    m.access = 'open' AND
    m.data_category = 'Transcriptome Profiling' AND
    m.project_id =  'TCGA-CHOL'
ORDER BY m.project_id , f.filename
"""

rows  = query( sql )

cnt = 0
pre_project_id = ""

for row in  rows :
    cnt += 1
    if cnt > 0  :
        cmd = "curl --remote-name --remote-header-name 'https://gdc-api.nci.nih.gov/data/%s' "  %( row['id']  )
        print( cmd )
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()

        project_id = row['project_id']
        file_name = row['filename']

        if pre_project_id != project_id :
            cmd = "hdfs dfs -mkdir -p  /data/tcga/rna-seq/%s" %( project_id )
            process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout, stderr = process.communicate()
            pre_project_id = project_id


        if os.path.exists(file_name):
            if file_name.find(".gz") != -1 :
                cmd = "gzip -d %s" % (file_name)
                process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                stdout, stderr = process.communicate()
                file_name = file_name.replace(".gz" , "" )

            cmd = "hdfs dfs -put  %s  /data/tcga/rna-seq/%s" %( file_name, project_id )
            process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout, stderr = process.communicate()
            os.remove( file_name )
            if cnt % 20 == 0 :
                print( "download cnt=", cnt  )
